# Deep Deterministic Policy Gradients (DDPG)
---
In this notebook, we train DDPG with OpenAI Gym's BipedalWalker-v2 environment.

### 1. Import the Necessary Packages

In [1]:
import gym
import numpy as np
import random
import copy
from collections import namedtuple, deque
from replay_buffer import ReplayBuffer

import tensorflow as tf
from keras import backend as K
from keras.layers import Dense, Input, Add, Activation,LeakyReLU
from keras.layers import GaussianNoise, Dropout, Concatenate
from keras.models import Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
%matplotlib inline

from ddpg_agent import Agent
from BFSWorld import BFSWorldEnv
from infoCalc import *

Using TensorFlow backend.


### 2. Instantiate the Environment and Agent

In [2]:
sess = tf.Session()
K.set_session(sess)

env = BFSWorldEnv()
env.seed(10)
print(env.action_space.shape[0])
agent = Agent(env, sess,state_size=env.observation_space.shape[0], action_size=env.action_space.shape[0], random_seed=10)
#agent.NetworkSummary()
#agent.load_network('savedModels/', 'saved.h5')

4
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


### 3. Train the Agent with DDPG

Run the code cell below to train the agent from scratch.  Alternatively, you can skip to the next code cell to load the pre-trained weights from file.

In [3]:
def ddpg(n_episodes=1000, max_t=30):
    scores_deque = deque(maxlen=1000)
    scores = []
    max_score = -np.Inf
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        #agent.reset()

        score = 0
        for t in range(max_t):
            #print(t,len(env.StateBuffer),state,score)
            action = agent.act(state,True)
            if(math.isnan(action[-1])):
                print("nan PrState, action",state,action)
            #print(t,len(env.StateBuffer),state,score,action)
            next_state, reward, done, _ = env.step(action)
            
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            #print(score)
            #env.render()
            if done:
                #print("done",done)
                break 
        #env.render()
        scores_deque.append(score)
        scores.append(score)
        #env.render()
        print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(i_episode, np.mean(scores_deque), score), end="")
        if i_episode % 1000 == 0:
            agent.save_network('savedModels/', 'saved.h5')
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            env.render()
        if i_episode % 3000 == 0:
            env.render()
        
        
        agent.save_network('savedModels/', 'saved.h5')
    return scores

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 7	Average Score: 105.84	Score: -5.005[array([[-0.65592876,  4.        ,  2.0682404 ,  6.        ],
       [-1.        ,  4.        ,  1.06048544,  6.        ],
       [-1.        ,  4.        ,  2.27257198,  6.        ],
       [-1.        ,  4.        ,  1.09101085,  1.80756567],
       [-1.        ,  4.        ,  2.0682404 ,  6.        ],
       [-0.65592876,  4.        ,  2.0682404 ,  2.79673374],
       [-1.        ,  4.        ,  1.11336131,  2.27257198],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-0.52218339,  4.        ,  2.79673374,  6.        ],
       [-1.        , -0.99496943,  1.09101085,  1.80756567],
       [-0.65592876,  4.        ,  2.0682404 ,  6.        ],
       [-1.        ,  4.        ,  2.0682404 ,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [ 0.17571533,  4.        ,  1.  

Episode 8	Average Score: 95.01	Score: 19.19[array([[-0.65592876,  4.        ,  2.79673374,  6.        ],
       [-1.        ,  4.        ,  1.09101085,  1.80756567],
       [-1.        ,  4.        ,  1.06048544,  6.        ],
       [-1.        ,  4.        ,  2.27257198,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-0.65592876,  4.        ,  2.0682404 ,  6.        ],
       [-0.65592876,  4.        ,  2.0682404 ,  6.        ],
       [-0.65592876,  4.        ,  2.0682404 ,  2.79673374],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-0.52218339,  4.        ,  2.79673374,  6.        ],
       [ 0.17571533,  4.        ,  1.        ,  1.52996688],
       [-0.99496943,  4.        ,  1.09101085,  1.80756567],
       [-1.        ,  4.        ,  1.52996688,  2.69237092],
       [-1.        , -0.65592876,  4.68203163,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.0910

[array([[-1.        ,  4.        ,  1.        ,  1.11336131],
       [-0.65592876,  4.        ,  2.0682404 ,  2.79673374],
       [-1.        ,  4.        ,  1.06048544,  6.        ],
       [-0.52218339,  4.        ,  2.79673374,  6.        ],
       [-0.65592876,  4.        ,  2.0682404 ,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.06533591,  1.74115083],
       [-1.        ,  4.        ,  2.69237092,  6.        ],
       [-1.        ,  4.        ,  1.04928621,  1.74115083],
       [-0.99496943,  4.        ,  1.26495535,  1.80756567],
       [-1.        ,  4.        ,  1.80756567,  6.        ],
       [-1.        , -0.65592876,  4.68203163,  6.        ],
       [ 0.17571533,  4.        ,  1.        ,  1.52996688],
       [-0.65592876,  4.        ,  2.0682404 ,  6.        ],
       [-1.        ,  4.        ,  1.04928621,  1.74115083],
       [-0.65592876,  4.        ,  2.79673374,  6.        ],
       [-1.        ,  4

Episode 9	Average Score: 79.73	Score: -42.52[array([[-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.04928621,  1.74115083],
       [-0.52218339,  4.        ,  2.79673374,  6.        ],
       [-1.        ,  4.        ,  2.0682404 ,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.        ,  2.02341019],
       [-1.        ,  4.        ,  1.74115083,  2.02341019],
       [-1.        ,  4.        ,  1.06048544,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-0.99496943,  4.        ,  1.09101085,  1.80756567],
       [ 0.17571533,  4.        ,  1.        ,  1.52996688],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.06533591,  1.74115083],
       [-0.65592876,  4.        ,  2.0682404 ,  6.        ],
       [-1.        ,  4.        ,  1.   

Episode 10	Average Score: 60.75	Score: -110.10[array([[-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.        ,  1.52996688],
       [-1.        ,  4.        ,  2.69237092,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.09101085,  1.80756567],
       [-1.        ,  4.        ,  2.02341019,  6.        ],
       [-1.        ,  4.        ,  1.52996688,  2.69237092],
       [-1.        ,  4.        ,  1.06048544,  6.        ],
       [-1.        ,  4.        ,  1.        ,  2.02341019],
       [-1.        ,  4.        ,  2.0682404 ,  6.        ],
       [ 0.17571533,  4.        ,  1.        ,  1.52996688],
       [-1.        ,  4.        ,  1.52996688,  6.        ],
       [-0.99496943,  4.        ,  1.26495535,  1.80756567],
       [-1.        ,  4.        ,  1.04928621,  1.74115083],
       [-0.57174793,  4.        ,  1. 

...

[array([[-1.        ,  4.        ,  2.17930904,  2.70047462],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.13368523,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-0.60914478,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  2.69237092,  6.        ],
       [-0.09109913,  4.        ,  5.42669839,  6.        ],
       [-1.        ,  4.        ,  5.16049153,  6.        ],
       [-1.        ,  4.        ,  3.3721987 ,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  0.37581557,  3.11797044,  5.03094023],
       [-1.        ,  4.        ,  1.        ,  2.75435632],
       [-1.        ,  4.        ,  2.83210298,  6.        ],
       [-1.        ,  4.        ,  1.        ,  3.69484967],
       [ 2.25779974,  4.        ,  1.        ,  1.50851189],
       [-0.4508557 ,  4

Episode 188	Average Score: -106.80	Score: -9.20[array([[-0.65175871,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  4.76171499,  6.        ],
       [-1.        ,  4.        ,  2.0682404 ,  6.        ],
       [-0.65592876,  4.        ,  2.0682404 ,  6.        ],
       [-0.99496943,  4.        ,  1.09101085,  1.80756567],
       [-1.        ,  4.        ,  3.36080408,  4.26261371],
       [ 1.41874158,  4.        ,  1.52808587,  5.94619548],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  3.0095183 ,  5.71170485],
       [-0.60914478,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  5.84208316,  6.        ],
       [-1.        ,  4.        ,  1.80756567,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.9086296 ,  2.01656191],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.

Episode 190	Average Score: -106.13	Score: -79.72[array([[-0.03294146,  4.        ,  1.04347258,  1.62065046],
       [-0.05562557,  4.        ,  2.26514152,  3.71618992],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [ 2.79198641,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.52808587,  6.        ],
       [-1.        ,  4.        ,  3.86776155,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [ 1.98042297,  4.        ,  1.        ,  2.02100506],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.28082233,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.36484506,  6.        ],
       [-0.99496943,  4.        ,  1.09101085,  1.26495535],
       [-1.        ,  4.        ,  1

Episode 191	Average Score: -106.06	Score: -94.29[array([[-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        , -0.65592876,  4.68203163,  6.        ],
       [-0.52819776,  4.        ,  2.82550728,  3.3721987 ],
       [-0.4508557 ,  4.        ,  1.        ,  6.        ],
       [ 2.79198641,  4.        ,  1.        ,  4.75260115],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-0.60914478,  4.        ,  1.        ,  6.        ],
       [ 3.46980178,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.        ,  3.3721987 ],
       [-1.        ,  4.        ,  1.        ,  4.70528311],
       [ 2.26055169,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  5.5752728 ,  5.64413762],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-1.        ,  4.        ,  1.        ,  6.        ],
       [-0.65592876,  4.        ,  2

KeyboardInterrupt: 

### 4. Watch a Smart Agent!

In the next code cell, you will load the trained weights from file to watch a smart agent!

In [ ]:
agent.load_network('savedModels/', 'saved.h5')
state = env.reset()
   
while True:
    action = agent.act(state,False)
    env.render()
    next_state, reward, done, _ = env.step(action)
    print(reward)
    state = next_state
    if done:
        break
        
env.close()

### 5. Explore

In this exercise, we have provided a sample DDPG agent and demonstrated how to use it to solve an OpenAI Gym environment.  To continue your learning, you are encouraged to complete any (or all!) of the following tasks:
- Amend the various hyperparameters and network architecture to see if you can get your agent to solve the environment faster than this benchmark implementation.  Once you build intuition for the hyperparameters that work well with this environment, try solving a different OpenAI Gym task!
- Write your own DDPG implementation.  Use this code as reference only when needed -- try as much as you can to write your own algorithm from scratch.
- You may also like to implement prioritized experience replay, to see if it speeds learning.  
- The current implementation adds Ornsetein-Uhlenbeck noise to the action space.  However, it has [been shown](https://blog.openai.com/better-exploration-with-parameter-noise/) that adding noise to the parameters of the neural network policy can improve performance.  Make this change to the code, to verify it for yourself!
- Write a blog post explaining the intuition behind the DDPG algorithm and demonstrating how to use it to solve an RL environment of your choosing.  